In [1]:
import numpy as np
import pandas as pd

In [261]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cafeteria/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cafeteria/test.csv')

In [262]:
all_data = pd.concat([train, test], sort=False)
#all_data = train
all_data.count()

일자                1255
요일                1255
본사정원수             1255
본사휴가자수            1255
본사출장자수            1255
본사시간외근무명령서승인건수    1255
현본사소속재택근무자수       1255
조식메뉴              1255
중식메뉴              1255
석식메뉴              1255
중식계               1205
석식계               1205
dtype: int64

In [263]:
all_data['year'] = pd.to_numeric(all_data['일자'].str[:4])
all_data['month'] = pd.to_numeric(all_data['일자'].str[5:7])
all_data['day'] = pd.to_numeric(all_data['일자'].str[8:10])


weeks = ['월', '화', '수', '목', '금']
for i in range(0, 5):
  all_data.loc[all_data['요일'] == weeks[i], 'wDay'] = i+1

In [264]:
all_data = all_data.drop('일자', axis=1)
all_data = all_data.drop('요일', axis=1)

In [265]:
all_data['left_members_at_lunch'] = all_data['본사정원수'] - all_data['본사휴가자수'] - all_data['본사출장자수'] - all_data['현본사소속재택근무자수']
all_data['left_members_at_dinner'] = all_data['left_members_at_lunch'] + all_data['본사시간외근무명령서승인건수']


In [266]:
all_data = all_data.drop('본사정원수', axis=1)
all_data = all_data.drop('본사휴가자수', axis=1)
all_data = all_data.drop('본사출장자수', axis=1)
all_data = all_data.drop('본사시간외근무명령서승인건수', axis=1)
all_data = all_data.drop('현본사소속재택근무자수', axis=1)

In [268]:
arr = ['조식메뉴', '중식메뉴', '석식메뉴']

for j in arr:
  all_data[j] = all_data[j].replace('  ', ' ')

In [269]:
all_data.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner
count,1205.000000,1205.000000,1255.000000,1255.000000,1255.00000,1255.000000,1255.000000,1255.000000
mean,890.334440,461.772614,2018.169721,6.358566,15.89243,3.004781,2363.448606,2641.789641
std,209.505057,139.179202,1.518847,3.470845,8.67904,1.414205,173.980270,346.539027
min,296.000000,0.000000,2016.000000,1.000000,1.00000,1.000000,1372.000000,1417.000000
25%,758.000000,406.000000,2017.000000,3.000000,8.00000,2.000000,2280.500000,2355.000000
50%,879.000000,483.000000,2018.000000,6.000000,16.00000,3.000000,2355.000000,2640.000000
75%,1032.000000,545.000000,2019.000000,9.000000,23.00000,4.000000,2454.500000,2882.000000
max,1459.000000,905.000000,2021.000000,12.000000,31.00000,5.000000,2921.000000,3548.000000


In [270]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

all_data = train

In [271]:
all_data['dinner_rates'] = 0
#all_data.loc[(all_data['석식계'] > 0) and (all_data['left_members_at_dinner'] > 0), 'dinner_rates'] = all_data['석식계'] / all_data['left_members_at_dinner']
all_data['dinner_rates'] = all_data['석식계'] / all_data['left_members_at_dinner']
all_data.loc[all_data['석식계'] == 0, 'dinner_rates'] = 0
all_data.loc[np.isinf(all_data['dinner_rates']), 'dinner_rates'] = 10
all_data['lunch_rates'] = all_data['중식계'] / all_data['left_members_at_lunch']

all_data.describe()

#len(all_data.loc[all_data['석식계'] == 0, 'dinner_rates'])

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,890.334440,461.772614,2018.052282,6.512033,15.947718,3.003320,2365.253112,2639.370124,0.174429,0.376458
std,209.505057,139.179202,1.433958,3.453906,8.650452,1.415384,176.563062,344.510427,0.048603,0.085110
min,296.000000,0.000000,2016.000000,1.000000,1.000000,1.000000,1372.000000,1417.000000,0.000000,0.143649
25%,758.000000,406.000000,2017.000000,4.000000,9.000000,2.000000,2281.000000,2356.000000,0.156705,0.317741
50%,879.000000,483.000000,2018.000000,7.000000,16.000000,3.000000,2357.000000,2637.000000,0.181917,0.371933
75%,1032.000000,545.000000,2019.000000,9.000000,23.000000,4.000000,2461.000000,2876.000000,0.204009,0.432010
max,1459.000000,905.000000,2021.000000,12.000000,31.000000,5.000000,2921.000000,3548.000000,0.376926,0.597203


In [272]:
min = all_data['lunch_rates'].min()
max = all_data['lunch_rates'].max()
all_data['lunch_rates'] = (9 * (all_data['lunch_rates'] - min) / (max - min)) + 1

min = all_data.loc[all_data['dinner_rates'] > 0, 'dinner_rates'].min()
max = all_data.loc[all_data['dinner_rates'] > 0, 'dinner_rates'].max()
all_data.loc[all_data['dinner_rates'] > 0, 'dinner_rates'] = (9 * (all_data['dinner_rates'] - min) / (max - min)) + 1


#all_data = all_data.reset_index()
all_data.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,890.334440,461.772614,2018.052282,6.512033,15.947718,3.003320,2365.253112,2639.370124,4.495154,5.619696
std,209.505057,139.179202,1.433958,3.453906,8.650452,1.415384,176.563062,344.510427,1.290147,1.688861
min,296.000000,0.000000,2016.000000,1.000000,1.000000,1.000000,1372.000000,1417.000000,0.000000,1.000000
25%,758.000000,406.000000,2017.000000,4.000000,9.000000,2.000000,2281.000000,2356.000000,4.003091,4.454559
50%,879.000000,483.000000,2018.000000,7.000000,16.000000,3.000000,2357.000000,2637.000000,4.689645,5.529897
75%,1032.000000,545.000000,2019.000000,9.000000,23.000000,4.000000,2461.000000,2876.000000,5.291222,6.722016
max,1459.000000,905.000000,2021.000000,12.000000,31.000000,5.000000,2921.000000,3548.000000,10.000000,10.000000


In [273]:
# 중식 평점 리스트
meals = ['중식메뉴']
dump1 = []

for i in meals:
  for j in range(0, len(all_data)):
    tmp = all_data[i][j].split(' ')
    for k in tmp:
      if k == ' ' or k == '':
        continue
      dump1.append(k)

print(len(dump1))
mySet = list(set(dump1))
mySet2 = set()
print(len(mySet))

drop_list = ['수입산', '국내산', '중국산', '호주산', '밥', '김치', '깍두기']
drop_count = 0

for i in mySet:
  should_drop = False

  if i.startswith('(') and i.endswith(')'):
    should_drop = True

  for j in drop_list:
    if j in i:
     should_drop = True


  if should_drop == False:
    mySet2.add(i)

print(len(mySet2))
mySet2 = list(mySet2)

9859
1797
1541


In [274]:
# 석식 평점 리스트
meals = ['석식메뉴']#, '조식메뉴']
dump1 = []

for i in meals:
  for j in range(0, len(all_data)):
    if all_data['석식계'][j] == 0:
      continue

    tmp = all_data[i][j].split(' ')
    for k in tmp:
      k = k.replace(' ', '')
      if k == ' ' or k == '':
        continue
      dump1.append(k)

print(len(dump1))
mySet = list(set(dump1))
mySet3 = set()
print(len(mySet))

drop_list = ['수입산', '국내산', '중국산', '호주산', '밥', '김치', '깍두기', '자기계발의날', '가정의날', '자기개발의날', '＜자기', '날＞', '계발의']
drop_count = 0

for i in mySet:
  should_drop = False

  if i.startswith('(') and i.endswith(')'):
    should_drop = True

  for j in drop_list:
    if j in i:
     should_drop = True


  if should_drop == False:
    i = i.replace(' ', '')
    mySet3.add(i)

print(len(mySet3))
mySet3 = list(mySet3)

9229
1759
1516


In [275]:
lunch_menu = [[0 for col in range(len(mySet2))] for row in range(4)]
lunch_menu[0] = list(mySet2)
print(len(lunch_menu[0]))

dinner_menu = [[0 for col in range(len(mySet3))] for row in range(4)]
dinner_menu[0] = list(mySet3)
print(len(dinner_menu[0]))


1541
1516


In [276]:
#중식 평점 매기기

for j in range(0, len(all_data['중식메뉴'])):
  tmp = all_data['중식메뉴'][j].split(' ')
  for k in tmp:
    if k == '':
      continue

    for l in range(0, len(lunch_menu[0])):
      if lunch_menu[0][l] == '':
        continue
        
      if k == lunch_menu[0][l]:
        lunch_menu[1][l] += all_data['lunch_rates'][j]
        lunch_menu[2][l] += 1
        break

In [277]:
#석식 평점 매기기

for j in range(0, len(all_data['석식메뉴'])):
  tmp = all_data['석식메뉴'][j].split(' ')
  if all_data['석식계'][j] == 0:
    continue

  for k in tmp:
    k = k.replace(' ', '')
    if k == '':
      continue

    for l in range(0, len(dinner_menu[0])):
      if dinner_menu[0][l] == '':
        continue
        
      if k == dinner_menu[0][l]:
        
        dinner_menu[1][l] += all_data['dinner_rates'][j]
        dinner_menu[2][l] += 1
        break


In [278]:
for i in range(0, len(lunch_menu[3])):
  if lunch_menu[2][i] == 0:
    print(lunch_menu[0][i]+' '+(str)(lunch_menu[1][i])+' '+(str)(lunch_menu[2][i])+' '+(str)(lunch_menu[3][i]))
  else:
    lunch_menu[3][i] = lunch_menu[1][i] / lunch_menu[2][i]

for i in range(0, len(lunch_menu[3])):
  if lunch_menu[3][i] == 0:
    print(lunch_menu[0][i]+' '+(str)(lunch_menu[1][i])+' '+(str)(lunch_menu[2][i])+' '+(str)(lunch_menu[3][i]))


In [279]:
for i in range(0, len(dinner_menu[3])):
  if dinner_menu[2][i] == 0:
    dinner_menu[3][i] = 5
    print(dinner_menu[0][i]+' '+(str)(dinner_menu[1][i])+' '+(str)(dinner_menu[2][i])+' '+(str)(dinner_menu[3][i]))
  else:
    dinner_menu[3][i] = dinner_menu[1][i] / dinner_menu[2][i]

for i in range(0, len(dinner_menu[3])):
  if dinner_menu[3][i] == 0:
    print(dinner_menu[0][i]+' '+(str)(dinner_menu[1][i])+' '+(str)(dinner_menu[2][i])+' '+(str)(dinner_menu[3][i]))


In [289]:
all_data = pd.concat([train, test], sort=False)
all_data = all_data.reset_index()
all_data.head()

,index,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates
0,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2016,2,1,1.0,2401.0,2639.0,3.151322,6.736437
1,1,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2016,2,2,2.0,2378.0,2697.0,5.390059,5.384232
2,2,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2016,2,3,3.0,2365.0,2476.0,6.037716,6.682558
3,3,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2016,2,4,4.0,2277.0,2632.0,5.167578,6.672465
4,4,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2016,2,5,5.0,2142.0,2176.0,3.865548,6.718638


In [290]:
all_data['predict_lunch_rate'] = 0

for i in range(0, len(all_data['중식메뉴'])):
  rates = 0
  count = 0

  meals = all_data['중식메뉴'][i]
  arr = meals.split(' ')
  for j in arr:
    for k in range(0, len(lunch_menu[0])):
      if j == lunch_menu[0][k]:
        rates += lunch_menu[3][k]
        count += 1

  if count > 0 :
    rates *= 1000
    rates /= count
    all_data['predict_lunch_rate'][i] = rates
  else:
    print(meals+' '+(str)(rates))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [294]:
all_data['predict_dinner_rate'] = 0

for i in range(0, len(all_data['석식메뉴'])):
  rates = 0
  count = 0

  meals = all_data['석식메뉴'][i]
  arr = meals.split(' ')
  for j in arr:
    if j == '':
      continue
    for k in range(0, len(dinner_menu[0])):
      #if j in dinner_menu[0][k]:
      #  print('='+j+' '+dinner_menu[0][k])
      if j == dinner_menu[0][k] and dinner_menu[3][k] > 0:
        #print('@')
        rates += dinner_menu[3][k]
        count += 1

  #print(meals+' # '+(str)(rates)+' '+(str)(count))
  if count > 0 :
    rates *= 1000
    rates /= count
    all_data['predict_dinner_rate'][i] = rates
  #print(all_data['predict_dinner_rate'][i])
  else:
    print(meals+' '+(str)(rates))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


      자기계발의날       0
      *자기계발의날*       0
      가정의날       0
             0
             0
             0
             0
    자기계발의날         0
    자기계발의날         0
    자기개발의날         0
    자기계발의날         0
    자기개발의날         0
    자기개발의날         0
 ＜자기 계발의 날＞       0
흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치  0
흑미밥 짬뽕수제비 생선까스*타르타르D 더덕무침 쑥갓무침 포기김치  0


In [295]:
all_data.describe()

,index,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates,predict_lunch_rate,predict_dinner_rate
count,1255.000000,1205.000000,1205.000000,1255.000000,1255.000000,1255.00000,1255.000000,1255.000000,1255.000000,1205.000000,1205.000000,1255.000000,1255.000000
mean,578.992032,890.334440,461.772614,2018.169721,6.358566,15.89243,3.004781,2363.448606,2641.789641,4.495154,5.619696,5617.620717,4594.543426
std,359.235388,209.505057,139.179202,1.518847,3.470845,8.67904,1.414205,173.980270,346.539027,1.290147,1.688861,773.128233,699.336313
min,0.000000,296.000000,0.000000,2016.000000,1.000000,1.00000,1.000000,1372.000000,1417.000000,0.000000,1.000000,2548.000000,0.000000
25%,263.500000,758.000000,406.000000,2017.000000,3.000000,8.00000,2.000000,2280.500000,2355.000000,4.003091,4.454559,5136.000000,4436.500000
50%,577.000000,879.000000,483.000000,2018.000000,6.000000,16.00000,3.000000,2355.000000,2640.000000,4.689645,5.529897,5557.000000,4719.000000
75%,890.500000,1032.000000,545.000000,2019.000000,9.000000,23.00000,4.000000,2454.500000,2882.000000,5.291222,6.722016,6047.000000,4932.000000
max,1204.000000,1459.000000,905.000000,2021.000000,12.000000,31.00000,5.000000,2921.000000,3548.000000,10.000000,10.000000,9376.000000,6403.000000


In [296]:
for i in range(0, len(all_data['predict_lunch_rate'])):
  if all_data['predict_lunch_rate'][i] == 0:
    print('중식 : '+all_data['중식메뉴'][i])

for i in range(0, len(all_data['predict_dinner_rate'])):
  if all_data['predict_dinner_rate'][i] == 0:
    print('석식 : '+all_data['석식메뉴'][i])

석식 :       자기계발의날      
석식 :       *자기계발의날*      
석식 :       가정의날      
석식 :             
석식 :             
석식 :             
석식 :             
석식 :     자기계발의날        
석식 :     자기계발의날        
석식 :     자기개발의날        
석식 :     자기계발의날        
석식 :     자기개발의날        
석식 :     자기개발의날        
석식 :  ＜자기 계발의 날＞      
석식 : 흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치 
석식 : 흑미밥 짬뽕수제비 생선까스*타르타르D 더덕무침 쑥갓무침 포기김치 


In [297]:
all_data.corr()

,index,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates,predict_lunch_rate,predict_dinner_rate
index,1.000000,-0.112966,-0.224198,0.740970,0.167859,-0.001785,-0.009960,0.305875,0.255324,-0.405805,-0.213633,-0.112399,-0.302512
중식계,-0.112966,1.000000,0.508287,-0.078804,-0.154664,-0.097392,-0.731563,0.286810,0.529821,0.310187,0.954018,0.753772,0.211136
석식계,-0.224198,0.508287,1.000000,-0.194792,-0.127142,-0.185565,-0.312112,0.172373,0.496586,0.899364,0.476374,0.421812,0.565303
year,0.740970,-0.078804,-0.194792,1.000000,-0.166000,-0.038843,-0.007803,0.262600,0.271636,-0.372178,-0.178877,-0.073896,-0.366187
month,0.167859,-0.154664,-0.127142,-0.166000,1.000000,0.005385,0.001438,0.017775,0.008876,-0.150623,-0.158003,-0.074644,-0.050194
day,-0.001785,-0.097392,-0.185565,-0.038843,0.005385,1.000000,-0.000478,-0.029538,-0.083588,-0.179632,-0.093265,-0.043504,-0.098895
wDay,-0.009960,-0.731563,-0.312112,-0.007803,0.001438,-0.000478,1.000000,-0.253636,-0.531389,-0.081624,-0.683024,-0.542920,-0.070750
left_members_at_lunch,0.305875,0.286810,0.172373,0.262600,0.017775,-0.029538,-0.253636,1.000000,0.721655,-0.139445,-0.005636,0.002873,-0.028453
left_members_at_dinner,0.255324,0.529821,0.496586,0.271636,0.008876,-0.083588,-0.531389,0.721655,1.000000,0.080168,0.327263,0.310257,0.066912
dinner_rates,-0.405805,0.310187,0.899364,-0.372178,-0.150623,-0.179632,-0.081624,-0.139445,0.080168,1.000000,0.367826,0.306416,0.626791


In [300]:
all_data = all_data.drop('index', axis=1)
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

In [303]:
test.columns
test.describe()

,중식계,석식계,year,month,day,wDay,left_members_at_lunch,left_members_at_dinner,dinner_rates,lunch_rates,predict_lunch_rate,predict_dinner_rate
count,0.0,0.0,50.0,50.000000,50.000000,50.000000,50.000000,50.000000,0.0,0.0,50.000000,50.000000
mean,NaN,NaN,2021.0,2.660000,14.560000,3.040000,2319.960000,2700.100000,NaN,NaN,5730.600000,4159.420000
std,NaN,NaN,0.0,0.798212,9.335384,1.399125,81.427559,391.627908,NaN,NaN,939.393921,1006.059196
min,NaN,NaN,2021.0,1.000000,1.000000,1.000000,2068.000000,2068.000000,NaN,NaN,3268.000000,0.000000
25%,NaN,NaN,2021.0,2.000000,6.250000,2.000000,2271.250000,2331.500000,NaN,NaN,5147.250000,3941.750000
50%,NaN,NaN,2021.0,3.000000,15.000000,3.000000,2333.500000,2811.000000,NaN,NaN,5712.000000,4347.500000
75%,NaN,NaN,2021.0,3.000000,23.000000,4.000000,2372.250000,3021.750000,NaN,NaN,6258.500000,4660.250000
max,NaN,NaN,2021.0,4.000000,31.000000,5.000000,2448.000000,3450.000000,NaN,NaN,7468.000000,5433.000000


In [304]:
x_col0 = ['month', 'wDay', 'left_members_at_lunch', 'left_members_at_dinner', 'predict_lunch_rate', 'predict_dinner_rate']
x_col1 = ['year', 'month', 'day', 'wDay', 'left_members_at_lunch', 'left_members_at_dinner', 'predict_lunch_rate', 'predict_dinner_rate']

y_col0 = '중식계'
y_col1 = '석식계'

In [308]:
x_train0 = train[x_col0]
y_train0 = train[y_col0]

x_train1 = train[x_col1]
y_train1 = train[y_col1]

x_test0 = test[x_col0]

x_test1 = test[x_col1]

In [305]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import mean_squared_error

In [314]:
xgb = XGBRegressor(random_state=37)
gbr = GradientBoostingRegressor(random_state=37)
rfr = RandomForestRegressor(random_state=37)
lr = LinearRegression()
lgr = LogisticRegression(random_state=37)
dtc = DecisionTreeClassifier(random_state=37)


models = [xgb, gbr, rfr, lr, lgr]
names = ['XGBRegressor', 'gradientBossting', 'randomForest', 'linearRegression', 'LogisticRegression']

#models = [rfr, lr]
#names = ['randomForest', 'linearRegression']


for m, n in zip(models, names):
  m.fit(x_train0, y_train0)
  pred0 = m.predict(x_test0)

  leng = len(y_train0)
  if leng > len(pred0):
    leng = len(pred0)

  tot_mse = mean_squared_error(y_train0[:leng], pred0[:leng], squared=False)
  print(n+" : "+(str)(mean_squared_error(y_train0[:leng], pred0[:leng], squared=False))+"")


  m.fit(x_train1, y_train1)
  pred1 = m.predict(x_test1)

  leng = len(y_train1)
  if leng > len(pred1):
    leng = len(pred1)

  tot_mse += mean_squared_error(y_train1[:leng], pred1[:leng], squared=False)
  print(n+" : "+(str)(mean_squared_error(y_train1[:leng], pred1[:leng], squared=False))+"")
  print((str)(tot_mse)+'\r\n')

[08:00:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor : 242.19341754424747
[08:00:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor : 181.59943553443443
423.7928530786819

gradientBossting : 247.5748871614512
gradientBossting : 182.389087316542
429.9639744779932

randomForest : 263.5463295437825
randomForest : 188.56047623507956
452.10680577886205

linearRegression : 240.10078381944496
linearRegression : 186.8512649845856
426.9520488040306



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression : 288.66149725933315
LogisticRegression : 280.86591106789734
569.5274083272304



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [315]:
xgb.fit(x_train0, y_train0)
pred0 = xgb.predict(x_test0)

xgb.fit(x_train1, y_train1)
pred1 = xgb.predict(x_test1)

[08:01:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:01:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [319]:

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cafeteria/sample_submission.csv', index_col='일자')

#for i in range(0, len(submission)):
#  submission['중식계'][i] = pred[i][0]
#  submission['석식계'][i] = pred[i][1]

print(pred0.shape)

#pred0 = pred0.reshape(2, 50)
#pred1 = pred1.reshape(2, 50)

submission['중식계'] = pred0
submission['석식계'] = pred1

submission.head()

(50,)


,중식계,석식계
일자,,
2021-01-27,980.599060,230.717529
2021-01-28,1009.122559,509.587616
2021-01-29,737.504272,358.472656
2021-02-01,1227.522583,621.586853
2021-02-02,1110.846802,687.092712


In [320]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/cafeteria/submission.csv', index='일자')